In [ ]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import plotly.io as pio
pio.renderers.default = "browser"  # Force plots to open in the default browser

# Set granularity level for timeline visualization
granularity = "monthly"  # Options: daily / weekly / monthly / quarterly / yearly

# ⏱️ Define time axis formatting based on granularity
def get_time_settings(granularity):
    if granularity == "daily":
        return {"tickformat": "%d %b", "dtick": "D1"}
    elif granularity == "weekly":
        return {"tickformat": "Week %W\n%Y", "dtick": "M1"}
    elif granularity == "monthly":
        return {"tickformat": "%b %Y", "dtick": "M1"}
    elif granularity == "quarterly":
        return {"tickformat": "Q%q %Y", "dtick": "M3"}
    elif granularity == "yearly":
        return {"tickformat": "%Y", "dtick": "M12"}
    else:
        return {}

# 📥 Load ClickUp export and drop rows without date info
df = pd.read_excel("all_clickup_tasks_with_subtasks.xlsx")
df = df.dropna(subset=["start_date", "due_date"])

rows = []

# 🔁 Build visual hierarchy: Project → Task → Subtask
for list_name in df["list"].unique():
    group = df[df["list"] == list_name]
    start_proj = pd.to_datetime(group["start_date"]).min()
    end_proj = pd.to_datetime(group["due_date"]).max()
    rows.append({
        "y_label": f"📦 {list_name}",  # Top-level project block
        "start": start_proj,
        "end": end_proj,
        "level": 0,
        "project": list_name
    })

    tasks = group[group["type"] == "task"].sort_values("start_date")
    for _, task_row in tasks.iterrows():
        task_id = task_row["task_id"]
        rows.append({
            "y_label": f"   📌 {task_row['task_name']} [{task_row['status']}]",
            "start": pd.to_datetime(task_row["start_date"]),
            "end": pd.to_datetime(task_row["due_date"]),
            "level": 1,
            "project": list_name
        })

        subtasks = group[(group["type"] == "subtask") & (group["parent_id"] == task_id)].sort_values("start_date")
        for _, sub_row in subtasks.iterrows():
            rows.append({
                "y_label": f"      └🧷 {sub_row['task_name']} [{sub_row['status']}]",
                "start": pd.to_datetime(sub_row["start_date"]),
                "end": pd.to_datetime(sub_row["due_date"]),
                "level": 2,
                "project": list_name
            })

# 🧱 Convert rows to DataFrame for plotting
timeline_df = pd.DataFrame(rows)

# 📊 Build Plotly Gantt-style timeline
fig = px.timeline(
    timeline_df,
    x_start="start",
    x_end="end",
    y="y_label",
    color="project",
    title=f"🗓️ Timeline View ({granularity.capitalize()})"
)

# 🔁 Flip Y-axis to match Gantt chart layout
fig.update_yaxes(autorange="reversed")

# ⌛ Apply time formatting based on granularity
time_config = get_time_settings(granularity)
fig.update_xaxes(**time_config)

# 🎨 General layout adjustments
fig.update_layout(
    height=900,
    margin=dict(l=300, r=50, t=80, b=50),
    legend_title="Project"
)

fig.show()
